In [1]:
!pip install ../input/timm22nov/pytorch-image-models-master

Processing /kaggle/input/timm22nov/pytorch-image-models-master
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for timm: filename=timm-0.5.0-py3-none-any.whl size=424001 sha256=1d13307c2f5dd173a8b3f65e4e6c60137a0bcb915c136f18d62c3e188dc87147
  Stored in directory: /root/.cache/pip/wheels/c9/5e/a8/445d26613f8ec895f465af030f071dfdcaaebb123a0f6c1d73
Successfully built timm


In [2]:
# pytorch lib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import timm


# image agumentation lib
import albumentations as A
from albumentations.pytorch import ToTensorV2

# numeric lib
import numpy as np
import pandas as pd

# image library
from PIL import Image

#python lib
import random
import os
import tqdm

# ploting library
import matplotlib.pyplot as plt
import wandb 

#sklearn library
from sklearn.model_selection import StratifiedKFold

In [3]:
class Config:
    seed = 42
    epochs = 4
    train_img_dir = "../input/petfinder-pawpularity-score/train"
    image_size = 384
    n_splits = 10
    model_name = 'efficientnet_b3'
    train_batchsize = 32
    val_batchsize = 32
    debug = False
    
    fp16 = True
    
    #optimizer
    optimizer = "Adam"
    
    if optimizer == "Adam":
        optimizer_params = dict(
            lr = 1e-4,
            betas = (0.9, 0.999),
            eps = 1e-8,
            weight_decay = 0,
            amsgrad = False
        )
        
    elif optimizer == "SGD":
        optimizer_params = dict(
            lr = 1e-3,
            weight_decay = 0,
            dampening  = 0,
            nesterov = False
        ) 
        
    # Scheduler
    scheduler = 'CosineAnnealingLR' # CosineAnnealingLR, ReduceLROnPlateau
    if scheduler == "CosineAnnealingLR":
        scheduler_params = dict(
            T_max = epochs,
            eta_min = 0,
            last_epoch = -1,
            verbose = False
        )
    elif scheduler == "ReduceLROnPlateau":
        scheduler_params = dict(
            mode = "min",
            factor= 0.1,
            patience=4,
            threshold=1e-4,
            min_lr = 1e-6
        )

In [4]:
def seed_python(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
def seed_torch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
seed_python(Config.seed)
seed_torch(Config.seed)

In [5]:
class PawDataset(Dataset):
    def __init__(self, image_fps, targets=None, transforms=None):
        super(PawDataset, self).__init__()
        self.image_fps = image_fps
        self.transforms = transforms
        self.targets = targets
        
    def __len__(self):
        return len(self.image_fps)
    
    def __getitem__(self, idx):
        image_fp = self.image_fps[idx]
        img = np.array(Image.open(image_fp))
        img = self.transforms(image=img)['image']
        
        if self.targets is not None:
            target = torch.tensor(self.targets[idx]).float()
            return img, target
        
        else:
            return img


In [6]:
class PawModel(nn.Module):
    def __init__(self, model_name):
        super(PawModel, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=False, in_chans=3) #this will create a model with classifier
        self.fcn = nn.Linear(self.backbone.classifier.in_features, 128)
        self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(p=0.1)
        self.output = nn.Linear(128, 1)
        
    def forward(self,image_array):
        x = self.backbone(image_array)
        x = self.fcn(x)
        x = self.dropout(x)
        return self.output(x)
        
#Test
# model = PawModel()
# model(test_dataset[0][0].unsqueeze(0))

In [7]:
def get_test_loaders(test_ds, test_batchsize=Config.val_batchsize):
    test_transform = A.Compose([
    A.Resize(Config.image_size, Config.image_size,p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
    ToTensorV2()], p=1)
    
    test_ds = PawDataset(test_ds["filepath"].to_numpy(), targets=None, transforms=test_transform)
    test_dataloader = DataLoader(test_ds, batch_size=test_batchsize, shuffle=False, pin_memory=True)
    
    return test_dataloader


In [8]:
def get_predictions(model, test_dataloader,device):
    model.eval()
    model.to(device)
    progbar = tqdm.tqdm(test_dataloader,desc='test' ,total=len(test_dataloader))
    y_preds = []
    for img in progbar:
        img = img.to(device)
        with torch.no_grad():
            y_pred = model(img).cpu().numpy() # (b,pred)
        
        y_preds.append(y_pred)
        
    return np.concatenate(y_preds)
    

## Loading the test dataframe

In [9]:
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
test_df['filepath'] = test_df['Id'].map(lambda x : os.path.join("../input/petfinder-pawpularity-score/test", x+".jpg"))
test_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,filepath
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


## Generating the test_dataloader

In [10]:
test_dataloader = get_test_loaders(test_df)

## Loading the models

In [11]:
model_0 = PawModel(Config.model_name)
model_0.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_0_best.pth"))
model_1 = PawModel(Config.model_name)
model_1.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_1_best.pth"))
model_2 = PawModel(Config.model_name)
model_2.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_2_best.pth"))
model_3 = PawModel(Config.model_name)
model_3.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_3_best.pth"))
model_4 = PawModel(Config.model_name)
model_4.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_4_best.pth"))
model_5 = PawModel(Config.model_name)
model_5.load_state_dict(torch.load("../input/pawpularity-trained-model/efficientnet_b3_n_fold_5_best.pth"))


<All keys matched successfully>

## Making prediction

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
test_df['model_0_preds'] = get_predictions(model_0, test_dataloader, device)
test_df['model_1_preds'] = get_predictions(model_1, test_dataloader, device)
test_df['model_2_preds'] = get_predictions(model_2, test_dataloader, device)
test_df['model_3_preds'] = get_predictions(model_3, test_dataloader, device)
test_df['model_4_preds'] = get_predictions(model_4, test_dataloader, device)
test_df['model_5_preds'] = get_predictions(model_5, test_dataloader, device)

test_df['Pawpularity'] = 1/6* test_df['model_0_preds'] + \
                         1/6* test_df['model_1_preds'] + \
                         1/6* test_df['model_2_preds'] + \
                         1/6* test_df['model_3_preds'] + \
                         1/6* test_df['model_4_preds'] + \
                         1/6* test_df['model_5_preds'] 

test: 100%|██████████| 1/1 [00:00<00:00, 11.27it/s]


## Submission

In [13]:
test_df['Pawpularity'] = test_df['Pawpularity'].clip(lower=0.0, upper=100.0)

In [14]:
test_df[['Id', 'Pawpularity']].to_csv("submission.csv", index=False)
test_df[['Id', 'Pawpularity']].head(15)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,0.000000
1,43a2262d7738e3d420d453815151079e,0.169400
2,4e429cead1848a298432a0acad014c9d,0.000000
3,80bc3ccafcc51b66303c2c263aa38486,0.000000
4,8f49844c382931444e68dffbe20228f4,0.115022
5,b03f7041962238a7c9d6537e22f9b017,0.000000
6,c978013571258ed6d4637f6e8cc9d6a3,0.000000
7,e0de453c1bffc20c22b072b34b54e50f,0.000000
